In [1]:
import argparse, os, sys, datetime, glob, importlib, csv
from omegaconf import OmegaConf
from pytorch_lightning.trainer import Trainer
import torch

from ldm.util import instantiate_from_config

/Users/hongseokoh/miniforge3/envs/zero123/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = OmegaConf.load('/Users/hongseokoh/Documents/GitHub/zero123/zero123/configs/sd-objaverse-finetune-c_concat-256.yaml')
finetune_from = '/Users/hongseokoh/sd-image-conditioned-v2.ckpt'

model = instantiate_from_config(config.model)
model.cpu()

old_state = torch.load(finetune_from, map_location="cpu")

if "state_dict" in old_state:
    print(f"Found nested key 'state_dict' in checkpoint, loading this instead")
    old_state = old_state["state_dict"]

# Check if we need to port weights from 4ch input to 8ch
in_filters_load = old_state["model.diffusion_model.input_blocks.0.0.weight"]
new_state = model.state_dict()
in_filters_current = new_state["model.diffusion_model.input_blocks.0.0.weight"]
in_shape = in_filters_current.shape
if in_shape != in_filters_load.shape:
    input_keys = [
        "model.diffusion_model.input_blocks.0.0.weight",
        "model_ema.diffusion_modelinput_blocks00weight",
    ]
    
    for input_key in input_keys:
        if input_key not in old_state or input_key not in new_state:
            continue
        input_weight = new_state[input_key]
        if input_weight.size() != old_state[input_key].size():
            print(f"Manual init: {input_key}")
            input_weight.zero_()
            input_weight[:, :4, :, :].copy_(old_state[input_key])
            old_state[input_key] = torch.nn.parameter.Parameter(input_weight)

m, u = model.load_state_dict(old_state, strict=False)


LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.53 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


100%|███████████████████████████████████████| 890M/890M [00:51<00:00, 18.1MiB/s]


Found nested key 'state_dict' in checkpoint, loading this instead
Manual init: model.diffusion_model.input_blocks.0.0.weight


In [6]:
in_filters_current.shape

torch.Size([320, 8, 3, 3])

In [7]:
in_filters_load.shape

torch.Size([320, 4, 3, 3])

In [10]:
new_state["model_ema.diffusion_modelinput_blocks00weight"].shape

torch.Size([320, 8, 3, 3])

In [11]:
old_state["model_ema.diffusion_modelinput_blocks00weight"].shape

KeyError: 'model_ema.diffusion_modelinput_blocks00weight'